In [7]:
import pymongo
from pymongo import MongoClient
import os
from dotenv import load_dotenv

load_dotenv()
ATLAS_KEY = os.getenv("ATLAS_KEY")
client = MongoClient(ATLAS_KEY)
db = client["imbd-scrap"]
collection = db["movies"]

### 1) Quel est le film le plus long ?

In [8]:
longest_movie = collection.find_one(sort=[('time', -1)], projection={'title': 1, 'time': 1, '_id': 0})
print(longest_movie)

{'title': 'Gone with the Wind', 'time': 238}


### 2) Quels sont les 5 films les mieux notés ?

In [15]:
top_movies = collection.find({}, {'title': 1, 'score': 1}).sort('score', pymongo.DESCENDING).limit(5)

for movie in top_movies:
    print(movie['title'], "   Score: ",movie['score'])

The Shawshank Redemption    Score:  9.3
The Godfather    Score:  9.2
The Godfather Part II    Score:  9.0
12 Angry Men    Score:  9.0
The Lord of the Rings: The Return of the King    Score:  9.0


### 3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [21]:
actor = "Morgan Freeman"
count = collection.count_documents({"stars": actor})
print(f"{actor} a joué dans {count} films")

Morgan Freeman a joué dans 4 films


In [22]:
actor = "Tom Cruise"
count = collection.count_documents({"stars": actor})
print(f"{actor} a joué dans {count} films")

Tom Cruise a joué dans 1 films


### 4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [27]:
list_genre = ["Horror", "Drama", "Comedy"]

for genre in list_genre:
    print(f"Les 3 meilleurs films de la catégorie '{genre}' sont :")
    movies = collection.find({'genre': genre}).sort([('score', -1)]).limit(3)
    for movie in movies:
        print(movie['title'], movie['score'])
    print('\n')


Les 3 meilleurs films de la catégorie 'Horror' sont :
Psycho 8.5
Alien 8.5
The Shining 8.4


Les 3 meilleurs films de la catégorie 'Drama' sont :
The Shawshank Redemption 9.3
The Godfather 9.2
The Lord of the Rings: The Return of the King 9.0


Les 3 meilleurs films de la catégorie 'Comedy' sont :
Life Is Beautiful 8.6
Modern Times 8.5
The Intouchables 8.5




### 5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [31]:
# Récupération des 100 films les mieux notés
top_100_movies = collection.find().sort('score', pymongo.DESCENDING).limit(100)

# Initialisation des compteurs pour les films américains et français
nb_american_movies = 0
nb_french_movies = 0

# Parcours de la liste des 100 films pour compter les films américains et français
for movie in top_100_movies:
    if 'United States' in movie['country']:
        nb_american_movies += 1
    if 'France' in movie['country']:
        nb_french_movies += 1


# Affichage des résultats
print(f"Parmi les 100 films les mieux notés, {nb_american_movies}% sont américains et {nb_french_movies}% sont français.")


Parmi les 100 films les mieux notés, 73% sont américains et 12% sont français.


### 6) Quel est la durée moyenne d’un film en fonction du genre ?

In [32]:
pipeline = [
    {"$group": {"_id": "$genre", "avg_duration": {"$avg": "$time"}}},
    {"$project": {"genre": "$_id", "avg_duration": 1, "_id": 0}}
]

results = collection.aggregate(pipeline)

for result in results:
    print(result)


{'avg_duration': 169.33333333333334, 'genre': ['Adventure', 'Biography', 'Drama']}
{'avg_duration': 147.0, 'genre': ['Drama', 'Mystery', 'War']}
{'avg_duration': 178.8, 'genre': ['Action', 'Adventure', 'Drama']}
{'avg_duration': 109.5, 'genre': ['Comedy', 'Crime']}
{'avg_duration': 136.2, 'genre': ['Action', 'Adventure', 'Sci-Fi']}
{'avg_duration': 148.5, 'genre': ['Western']}
{'avg_duration': 121.0, 'genre': ['Action', 'Adventure']}
{'avg_duration': 136.0, 'genre': ['Crime', 'Sci-Fi']}
{'avg_duration': 68.0, 'genre': ['Comedy', 'Drama', 'Family']}
{'avg_duration': 122.0, 'genre': ['Horror']}
{'avg_duration': 137.0, 'genre': ['Biography', 'Crime', 'Drama']}
{'avg_duration': 137.0, 'genre': ['Action', 'Drama', 'Thriller']}
{'avg_duration': 132.0, 'genre': ['Action', 'Thriller']}
{'avg_duration': 106.0, 'genre': ['Animation', 'Drama', 'Fantasy']}
{'avg_duration': 136.0, 'genre': ['Action', 'Adventure', 'Mystery']}
{'avg_duration': 121.75, 'genre': ['Drama', 'Thriller']}
{'avg_duration': 

In [40]:
import pandas as pd

genres = collection.distinct('genre')
result = {}

for genre in genres:
    count = collection.count_documents({'genre': genre})
    total_duration = sum(movie['time'] for movie in collection.find({'genre': genre}))
    avg_duration = total_duration / count
    result[genre] = round(avg_duration, 2)

df = pd.DataFrame(list(result.items()), columns=['genre', 'durée moyenne (min)'])
print(df)


        genre  durée moyenne (min)
0      Action               133.98
1   Adventure               130.12
2   Animation               100.96
3   Biography               145.50
4      Comedy               105.60
5       Crime               132.50
6       Drama               134.58
7      Family               114.54
8     Fantasy               123.57
9   Film-Noir               110.00
10    History               152.11
11     Horror               120.60
12      Music               134.25
13    Musical               103.00
14    Mystery               124.61
15    Romance               118.48
16     Sci-Fi               131.70
17      Sport               128.50
18   Thriller               123.88
19        War               140.43
20    Western               153.86
